# Tips for faster querying

This notebook gives tips, shows examples on how to query MongoDB and, most importantly, examines different approaches based on their speed - all in pymongo, python driver for MongoDB. 

Furthemore, excluding APIs/drivers in other frameworks and languages, two most common ways to query MongoDB are:

    - command line interface (mongosh - MongoDB shell) or 
    - GUI named MongoDB Compass (that has embedded MongoDB shell). 

Some operations are possible/easier to perform in Compass/shell, thus there is an occasional need for these as well. Moreover, choise depends on just a preference.

This notebook is a reference to Telegram database with 30 million messages on 22.01.2024.

As we are working with large files, you might need to update notebook RAM configuration, if you manage to mistakenly print large files. If needed, check this [blog post](https://towardsdatascience.com/leveraging-the-power-of-jupyter-notebooks-26b4b8d7c622).

# 0. Introduction

Let's start by creating a client object, displaying databases, collections, and gathering information about them. 

In MongoDB, database has many collections, and each collection is represented by documents (BSONs i.e. binary JSONs).

In [46]:
# Function
import time


def get_speed(collection, query, projection):
    start_time = time.time()

    cursor = collection.find(query, projection)
    documents = list(cursor)

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Number of documents retrieved: {len(documents)}")
    print(f"Time taken to list() retrieved documents: {elapsed_time} seconds")

    return documents

In [52]:
from pymongo import MongoClient
import re
import time
from pprint import pprint
from datetime import datetime
import sys

# Connect to MongoDB, path and namings of dbs and collections are the same as the ones defined in the config-database.ini before the crawl was initiated
client = MongoClient("mongodb://localhost:27017/")

# Let's check all the databases
dbs = client.list_database_names()
print("Databases:", dbs)

Databases: ['Telegram-090124-crawler', 'Telegram-1-crawler', 'Telegram-9-crawler', 'Test', 'admin', 'config', 'local', 'test-size', 'testing-size']


In [2]:
# Let's see our collections
db = client["Telegram-090124-crawler"]
collections = db.list_collection_names()

print("Collections:", collections)

Collections: ['messages', 'participants', 'chats', 'network']


In [3]:
collections_info = db.command("listCollections")
pprint(collections_info, indent=1)

{'cursor': {'firstBatch': [{'idIndex': {'key': {'_id': 1},
                                        'name': '_id_',
                                        'v': 2},
                            'info': {'readOnly': False,
                                     'uuid': UUID('5f31b076-3332-406c-9a0d-d282667b15a7')},
                            'name': 'messages',
                            'options': {},
                            'type': 'collection'},
                           {'idIndex': {'key': {'_id': 1},
                                        'name': '_id_',
                                        'v': 2},
                            'info': {'readOnly': False,
                                     'uuid': UUID('602eb318-401e-471a-aa76-79c7cf8adbfa')},
                            'name': 'participants',
                            'options': {},
                            'type': 'collection'},
                           {'idIndex': {'key': {'_id': 1},
                             

In [4]:
# We can also acquire more info about collections
collection_details = db.command("collstats", "messages")
# print(collection_details)

print(f"Docuemnts count: {collection_details['count']}")
print(f"Collection size: {collection_details['size']}")
print(f"Average document size: {collection_details['avgObjSize']}")
print(f"Index size: {collection_details['totalIndexSize']}")

Docuemnts count: 29755431
Collection size: 59586089993
Average document size: 2002
Index size: 1164312576


## 0.1 Accessing documents - types of queries

There are many ways we can query our MongoDB database and access documents.

### Method collection_name.find_one()

In [5]:
# let's find one document from our collection messages, but first, let's initiate a variable for a connection
messages_collection = db["messages"]
one_document = messages_collection.find_one()

# We can check the structure of the document and assume uniformity with other documents
# Important: there are three types of message objects retrieved from telegram , check https://core.telegram.org/type/Message
pprint(one_document, indent=1)

{'_': 'Message',
 '_id': ObjectId('659db99fcb4aa618cff2a132'),
 'date': datetime.datetime(2023, 12, 15, 15, 11, 30),
 'edit_date': None,
 'edit_hide': False,
 'entities': [{'_': 'MessageEntityUrl', 'length': 20, 'offset': 2}],
 'forwards': None,
 'from_id': {'_': 'PeerUser', 'user_id': 5963929426},
 'from_scheduled': False,
 'fwd_from': None,
 'grouped_id': None,
 'id': 16,
 'legacy': False,
 'media': {'_': 'MessageMediaWebPage',
           'webpage': {'_': 'WebPage',
                       'attributes': [],
                       'author': None,
                       'cached_page': None,
                       'description': '"Communism was not created by the '
                                      'masses to overthrow the bankers, '
                                      'Communism was created by the bankers to '
                                      'overthrow and enslave the masses."',
                       'display_url': 't.me/thelastbattle20',
                       'document': 

In [7]:
# Next, we can apply find_one based on a query, if there are more than one document that match our query, only one will be retrieved
# Input parameters for a query are {"key": "value"}

# let's find the date and time of the same message from the previous cell
print(one_document["date"])

target_date = datetime(2023, 12, 15, 15, 11, 30)
query = {"date": target_date}

one_specific_document = messages_collection.find_one(query)
pprint(one_specific_document, indent=1)

2023-12-15 15:11:30
{'_': 'Message',
 '_id': ObjectId('659db99fcb4aa618cff2a132'),
 'date': datetime.datetime(2023, 12, 15, 15, 11, 30),
 'edit_date': None,
 'edit_hide': False,
 'entities': [{'_': 'MessageEntityUrl', 'length': 20, 'offset': 2}],
 'forwards': None,
 'from_id': {'_': 'PeerUser', 'user_id': 5963929426},
 'from_scheduled': False,
 'fwd_from': None,
 'grouped_id': None,
 'id': 16,
 'legacy': False,
 'media': {'_': 'MessageMediaWebPage',
           'webpage': {'_': 'WebPage',
                       'attributes': [],
                       'author': None,
                       'cached_page': None,
                       'description': '"Communism was not created by the '
                                      'masses to overthrow the bankers, '
                                      'Communism was created by the bankers to '
                                      'overthrow and enslave the masses."',
                       'display_url': 't.me/thelastbattle20',
               

In [8]:
# Let's find another document that has 'edit_date' not None
# Note: as field "edit_date" does not have an index, it starts iterating over all messages
# until it finds one with "edit_date" not being None
# This should be done cautiously as if there is one field without the value you are looking for,
# it will iterate over every document in a collection

edit_date_document = messages_collection.find_one({"edit_date": {"$ne": None}})
pprint(edit_date_document, indent=1)

{'_': 'Message',
 '_id': ObjectId('659db9a1cb4aa618cff2a14d'),
 'date': datetime.datetime(2022, 5, 11, 20, 34, 46),
 'edit_date': datetime.datetime(2022, 6, 4, 20, 52, 42),
 'edit_hide': True,
 'entities': [{'_': 'MessageEntityPre',
               'language': '',
               'length': 84,
               'offset': 0},
              {'_': 'MessageEntityBold', 'length': 7, 'offset': 84},
              {'_': 'MessageEntityTextUrl',
               'length': 8,
               'offset': 92,
               'url': 'https://www.discogs.com/it/master/789202-Musikkorps-Und-Soldatenchor-der-11-Panzer-Grenadier-Division-Leitung-Hauptmann-Hans-Frie%C3%9F-Wir-Singe'},
              {'_': 'MessageEntityHashtag', 'length': 6, 'offset': 100},
              {'_': 'MessageEntityHashtag', 'length': 8, 'offset': 107},
              {'_': 'MessageEntityHashtag', 'length': 4, 'offset': 116},
              {'_': 'MessageEntityTextUrl',
               'length': 7,
               'offset': 123,
               

### Method collection_name.find()

In [9]:
# .find() applied on a collection returns a cursor that needs to be iterated over

# Note: if not parametrized, cursor represents all documents from the collection,
# so do not try to print or do anything else with millions of documents without setting a limit, or simply just iterate with indexing

# Let's skip first 100 documents, and access next 4
documents = messages_collection.find().skip(100).limit(4)

for document in documents:
    pprint(document, indent=1)

# Access just a subset of the documents
for index, document in enumerate(documents):
    if index < 2:
        print(document)

{'_': 'Message',
 '_id': ObjectId('659db9a1cb4aa618cff2a19b'),
 'date': datetime.datetime(2019, 12, 30, 0, 37, 54),
 'edit_date': None,
 'edit_hide': False,
 'entities': [],
 'forwards': 0,
 'from_id': None,
 'from_scheduled': False,
 'fwd_from': None,
 'grouped_id': None,
 'id': 723,
 'legacy': False,
 'media': None,
 'media_unread': False,
 'mentioned': False,
 'message': 'German version',
 'noforwards': False,
 'out': False,
 'peer_id': {'_': 'PeerChannel', 'channel_id': 1123235006},
 'pinned': False,
 'post': True,
 'post_author': None,
 'reactions': None,
 'replies': None,
 'reply_markup': None,
 'reply_to': {'_': 'MessageReplyHeader',
              'forum_topic': False,
              'reply_to_msg_id': 715,
              'reply_to_peer_id': None,
              'reply_to_scheduled': False,
              'reply_to_top_id': None},
 'restriction_reason': [],
 'silent': False,
 'ttl_period': None,
 'via_bot_id': None,
 'views': 1917}
{'_': 'Message',
 '_id': ObjectId('659db9a1cb4aa618

In [10]:
# We can define a query to find messages that were forwarded
# Our query returns five messages that were forwarded to other chats
query = {"forwards": {"$nin": [None, 0]}}
forwarded_messages = messages_collection.find(query).limit(5)

# print(len(list(forwarded_messages)))
for message in forwarded_messages:
    pprint(message, indent=1)

{'_': 'Message',
 '_id': ObjectId('659db99fcb4aa618cff2a13e'),
 'date': datetime.datetime(2023, 7, 19, 8, 50, 35),
 'edit_date': None,
 'edit_hide': False,
 'entities': [],
 'forwards': 2,
 'from_id': {'_': 'PeerUser', 'user_id': 5963929426},
 'from_scheduled': False,
 'fwd_from': {'_': 'MessageFwdHeader',
              'channel_post': 2,
              'date': datetime.datetime(2023, 7, 5, 11, 37, 56),
              'from_id': {'_': 'PeerChannel', 'channel_id': 1976161109},
              'from_name': None,
              'imported': False,
              'post_author': None,
              'psa_type': None,
              'saved_from_msg_id': None,
              'saved_from_peer': None},
 'grouped_id': None,
 'id': 3,
 'legacy': False,
 'media': None,
 'media_unread': False,
 'mentioned': False,
 'message': 'This is a test messages.',
 'noforwards': False,
 'out': True,
 'peer_id': {'_': 'PeerChannel', 'channel_id': 1318919825},
 'pinned': False,
 'post': False,
 'post_author': None,
 'rea

In [11]:
# We can also retrieve only fields that we need by defining "projection",
# Projection is a second arguemnt it defines what fields we want to retrieve

# Let's first filter messages that were forwarded to other chats,
# but let's only retrieve message text, id of the user that send it to current chat, and id of channel that it came from, if it came from somewhere
cursor = messages_collection.find(
    {"forwards": {"$nin": [None, 0]}},
    {
        "message": 1,
        "forwards": 1,
        "fwd_from.from_id.channel_id": 1,
        "from_id.user_id": 1,
    },
).limit(10)

for document in cursor:
    pprint(document, indent=2)

# To wrap up: we filtered only 10 messages that were forwarded to other chats, but from a whole message, we recieved
# fields that says from which channel the message was forwarded to the current channel as well as

{ '_id': ObjectId('659db99fcb4aa618cff2a13e'),
  'forwards': 2,
  'from_id': {'user_id': 5963929426},
  'fwd_from': {'from_id': {'channel_id': 1976161109}},
  'message': 'This is a test messages.'}
{ '_id': ObjectId('659db99fcb4aa618cff2a13f'),
  'forwards': 2,
  'from_id': {'user_id': 5963929426},
  'fwd_from': {'from_id': {'channel_id': 1976161109}},
  'message': 'This is a test messages.'}
{ '_id': ObjectId('659db9a1cb4aa618cff2a14d'),
  'forwards': 17,
  'message': 'Musikkorps und Soldatenchor der 11. Panzergrenadierdivision - '
             'Alte Kameraden, 1961\n'
             '\n'
             'Source: discogs\n'
             '#music #germany #brd 〜 agartha'}
{ '_id': ObjectId('659db9a1cb4aa618cff2a14e'),
  'forwards': 9,
  'message': 'I. Musikkorps des Infanterie-Regiments "Groß-Deutschland" - Alte '
             'Kameraden, 1936\n'
             '\n'
             'Source: InternetArchive\n'
             '#music #germany #nsdap 〜 agartha'}
{ '_id': ObjectId('659db9a1cb4aa618cff2

### Methods .count() and .distinct()

With these methods we can count number of documents based on criterion, or gather distinct values for the field. 

Here we are going to work on another collection as these two methods can be time consuming if the fields are not indexed.

We can observe, that when using the field "chat_id", the query is 60 times faster compared to using an unindexed field "phone". One can argue that for instance, number of distnict phones is 2.5 times larger than number of chat_ids, but on the other hand, here we shown that even for the field "contact" with only one value , querying by chat_id is more than 10 times faster!

We can see similar results with .count as well. 

Importance of indexing will be discussed in the next section.

In [12]:
participants_collection = db["participants"]

# participants_number = participants_collection.count_documents({})
# print(participants_number)
start_time = time.time()

distinct_chats = participants_collection.distinct("chat_id")
print(f"Number of distinct chats: {len(distinct_chats)}")
end_time = time.time()
elapsed_time = end_time - start_time


# Print the distinct values and the elapsed time
print(f"Time taken: {elapsed_time} seconds")

start_time = time.time()

distinct_phones = participants_collection.distinct("phone")
print(f"Number of distinct phones: {len(distinct_phones)}")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time} seconds")

start_time = time.time()

distinct_contacts = participants_collection.distinct("contact")
print(f"Number of distinct contacts: {len(distinct_contacts)}")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time} seconds")

Number of distinct chats: 140
Time taken: 0.003003835678100586 seconds
Number of distinct phones: 458
Time taken: 0.1979968547821045 seconds
Number of distinct contacts: 1
Time taken: 0.03899979591369629 seconds


In [13]:
# print chat_id for the next
print(distinct_chats[-10])

1826873627


In [28]:
start_time = time.time()
participants_number = participants_collection.count_documents({"chat_id": 1826873627})
print(f"Number of participants based on chat ids: {(participants_number)}")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time} seconds")

start_time = time.time()
participants_number = participants_collection.count_documents(
    {"phone": "4367764863157"}
)
print(f"Number of participants based on contact: {(participants_number)}")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time} seconds")

Number of participants based on chat ids: 28
Time taken: 0.0030121803283691406 seconds
Number of participants based on contact: 3
Time taken: 0.04696202278137207 seconds


### Aggregation

Here we use another way of querying MongoDB called aggregation. This can in some cases speed up the process as it can parallelize some filters from the piple line. Nevertheless, sometimes, it is not a good idea. 

In [31]:
# Aggregation pipeline for counting based on chat_id
chat_id_pipeline = [
    {"$match": {"chat_id": 1826873627}},
    {"$count": "participants_count"},
]

# Aggregation pipeline for counting based on phone
stories_unavailable_pipeline = [
    {"$match": {"phone": "4367764863157"}},
    {"$count": "participants_count"},
]

start_time = time.time()
result_chat_id = list(participants_collection.aggregate(chat_id_pipeline))
participants_count_chat_id = (
    result_chat_id[0]["participants_count"] if result_chat_id else 0
)
end_time = time.time()
elapsed_time_chat_id = end_time - start_time

start_time = time.time()
result_stories_unavailable = list(
    participants_collection.aggregate(stories_unavailable_pipeline)
)
participants_count_stories_unavailable = (
    result_stories_unavailable[0]["participants_count"]
    if result_stories_unavailable
    else 0
)
end_time = time.time()
elapsed_time_stories_unavailable = end_time - start_time

print(f"Number of participants based on chat_id: {participants_count_chat_id}")
print(f"Time taken for chat_id: {elapsed_time_chat_id} seconds")

print(
    f"Number of participants based on stories_unavailable: {participants_count_stories_unavailable}"
)
print(f"Time taken for stories_unavailable: {elapsed_time_stories_unavailable} seconds")

Number of participants based on chat_id: 28
Time taken for chat_id: 0.0019989013671875 seconds
Number of participants based on stories_unavailable: 3
Time taken for stories_unavailable: 0.0470128059387207 seconds


## 1. Indexing

Indexing can help a lot to speed up the query process. Some of the indices are created on the fly - while storing the data. Other indices can be created later on the existing collection. 

There are two types of indices:

    - Single index is made on one field - thus querying based on that field will be optimized
    - Compound index is created on two fields - this means we get optimal workflow on queries based on those two fields in the same time
        - Compound index does not work optimally for querying based on just the second indexed field, although querying based on just the first indexed field, will provide faster data retrieval compared to no index

Firstly, we are going to print indices for each collection. 

    - We see that collections "network" and "chats" does not have any indices beside the ones created automatically by MongoDB on ID field to distingish between them inside of a collection.
    - Collection "participants" has single index on the field "chat_id"
    - Collection "messages" has compound index on fields "date" and "peer_id.channel_id" - in that order! 

Then we make queries and examine their speed.

    - 

In [50]:
# Connect to MongoDB, path and namings are the same as the ones defined in the config-database.ini before the crawl was initiated
client = MongoClient("mongodb://localhost:27017/")
db = client["Telegram-090124-crawler"]
network_collection = db["network"]
messages_collection = db["messages"]
chats_collection = db["chats"]
participants_collection = db["participants"]

In [2]:
# List of collections
collections = ["network", "messages", "chats", "participants"]


# Function to print existing indexes for a collection
def print_indexes(collection_name):
    collection = db[collection_name]
    existing_indexes = collection.list_indexes()

    print(f"Indexes for collection '{collection_name}':")

    for index in existing_indexes:
        print(index)
    print("\n")


# Check indexes for each collection
for collection_name in collections:
    print_indexes(collection_name)

Indexes for collection 'network':
SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])


Indexes for collection 'messages':
SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('date', 1), ('peer_id.channel_id', 1)])), ('name', 'date_1_peer_id.channel_id_1')])


Indexes for collection 'chats':
SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])


Indexes for collection 'participants':
SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('chat_id', 1)])), ('name', 'chat_id_1')])




In [38]:
date = datetime(2024, 1, 1)
cursor_date = messages_collection.find({"date": date})

print(len(list(cursor_date)))

0


### Compound index on date and chat_id field

Now let's examine Compound index, but just on a "date" field, thus we are going to look at it as a single one. 

Firstly, let's choose a date or two, to retrieve messages. Cursor object, unless iterated or turned into a list, will not take anytime as we just initiated it. Thus, we need to measure an aciton on a cursor.

We see that choosing a date as our filter, such as Jan 1 2023, it takes around 7 second to retrieve the data.

In [ ]:
def get_speed(collection, query, projection):
    start_time = time.time()

    cursor = collection.find(query, projection)

    documents = list(cursor)

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Number of documents retrieved: {len(documents)}")
    print(f"Time taken to list() retrieved documents: {elapsed_time} seconds")

    return documents

In [53]:
start_time = time.time()

date_start = datetime(2023, 1, 1, 0, 0, 0)
date_end = datetime(2023, 1, 2, 0, 0, 0)

cursor_date = messages_collection.find({"date": {"$gte": date_start, "$lte": date_end}})
print(f"Messages num: {len(list(cursor_date))}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken to list(): {elapsed_time} seconds")


start_time = time.time()

cursor_date = messages_collection.find({"date": {"$gte": date_start, "$lte": date_end}})
mes_list = []
for item in cursor_date:
    mes_list.append(item)


print(f"Messages num: {len(list(mes_list))}")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken to iterate to a list: {elapsed_time} seconds")

Messages num: 18991
Time taken to list(): 6.92830228805542 seconds
Messages num: 18991
Time taken to iterate to a list: 6.578845739364624 seconds


Let's try to filter now on an unindexed field "chat_id" (which is part of compund index, but second in order thus cannot be used). Now it iterates over every document in a collection to find the ones that belong to this chat_id. We see that it took almost 4 minutes for 30 million documents. Now, If our db become considerably larger (let's say 10 times in the next three weeks depending on the number of acocunts scraping), gathering information on these 15 chats would take much more time.

Nevertheless, we see that if use 15 chat_ids to filter, query wil not last much longer. 

Important thing is that sorting cannot be done on fields that are not indexed.

In [63]:
start_time = time.time()
cursor_date = messages_collection.find({"peer_id.channel_id": 1826873627})
print(f"Messages num: {len(list(cursor_date))}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken to list all messages: {elapsed_time} seconds")

Messages num: 355
Time taken to list all messages: 224.6608145236969 seconds


In [84]:
# let's try 15 chats

chats_collections = db["chats"]
cursor_channels = chats_collections.find({}, {"full_chat.id"}).limit(15)
channel_ids = []

for item in cursor_channels:
    channel_ids.append(item.get("full_chat", {}).get("id"))

start_time = time.time()

cursor_date = messages_collection.find({"peer_id.channel_id": {"$in": channel_ids}})

print(f"Messages num: {len(list(cursor_date))}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken to list() 2024: {elapsed_time} seconds")

Messages num: 19616
Time taken to list() 2024: 250.95547032356262 seconds


Let's try to apply the same query with date and chat_id. Rememeber our compund index is on date,chat_id - in that order. 

We see it took considerably less time as it was able to sort our chat_ids after defining datetime filter. Nevertheless, One can argue that it would be more beneficial to have separate index on chat_id, or even reversed compund, depending on a usecase.

In [64]:
start_time = time.time()

date_start = datetime(2020, 1, 1, 0, 0, 0)
date_end = datetime(2024, 1, 1, 0, 0, 0)

cursor_date = messages_collection.find(
    {"date": {"$gte": date_start, "$lte": date_end}, "peer_id.channel_id": 1826873627}
)
print(f"Messages num: {len(list(cursor_date))}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken to list() 2024: {elapsed_time} seconds")

Messages num: 304
Time taken to list() 2024: 48.155343770980835 seconds


In [79]:
# Now let's try more channel_ids, we can find channels in chats colletction
chats_collections = db["chats"]
cursor_channels = chats_collections.find({}, {"full_chat.id"}).limit(15)
channel_ids = []

for item in cursor_channels:
    channel_ids.append(item.get("full_chat", {}).get("id"))

date_start = datetime(2020, 1, 1, 0, 0, 0)
date_end = datetime(2024, 1, 1, 0, 0, 0)

start_time = time.time()
cursor_date = messages_collection.find(
    {
        "date": {"$gte": date_start, "$lte": date_end},
        "peer_id.channel_id": {"$in": channel_ids},
    }
)
print(f"Messages num: {len(list(cursor_date))}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken to list() 2024: {elapsed_time} seconds")

Messages num: 11279
Time taken to list() 2024: 33.49662733078003 seconds


In [92]:
# let's skip frist 15, and use next 15, but also show that the order of fields does not matter, also let's now use 8 years
chats_collections = db["chats"]
cursor_channels = chats_collections.find({}, {"full_chat.id"}).skip(15).limit(15)
channel_ids = []

for item in cursor_channels:
    channel_ids.append(item.get("full_chat", {}).get("id"))

date_start = datetime(2016, 1, 1, 0, 0, 0)
date_end = datetime(2024, 1, 1, 0, 0, 0)

start_time = time.time()
cursor_date = messages_collection.find(
    {
        "peer_id.channel_id": {"$in": channel_ids},
        "date": {"$gte": date_start, "$lte": date_end},
    }
)
print(f"Messages num: {len(list(cursor_date))}")

end_time = time.time()
elapsed_time = end_time - start_time
print(
    f"Time taken to list() 15 chats based on datetime criteria: {elapsed_time} seconds"
)

Messages num: 32305
Time taken to list() 15 chats based on datetime criteria: 60.304965019226074 seconds


Finally, let's try date with another field that is not chat_id. We are going to parse messages and find some messages that were forwarded from another chat, and extract that chat. One might argue we could use aggregation pipeline for this query as well. 

We see that our query is still very slow. We can easily access all of the dates, but it takes time to iterate over non-indexed field.

In [17]:
cursor = messages_collection.find(
    {"fwd_from.from_id.channel_id": {"$ne": None}}, {"fwd_from.from_id.channel_id": 1}
).limit(20)

fwd_list = set()
for document in cursor:
    fwd_list.add(document.get("fwd_from", {}).get("from_id", {}).get("channel_id"))
fwd_list = list(fwd_list)
print(len(fwd_list))

date_start = datetime(2016, 1, 1, 0, 0, 0)
date_end = datetime(2024, 1, 1, 0, 0, 0)

start_time = time.time()
cursor_date = messages_collection.find(
    {
        "fwd_from.from_id.channel_id": {"$in": fwd_list},
        "date": {"$gte": date_start, "$lte": date_end},
    }
)
print(f"Messages num: {len(list(cursor_date))}")

end_time = time.time()
elapsed_time = end_time - start_time
print(
    f"Time taken to list() 15 chats based on datetime criteria: {elapsed_time} seconds"
)

5
Messages num: 4982
Time taken to list() 15 chats based on datetime criteria: 787.9138038158417 seconds


In the next cell, we parmetrized our query with the date in front of fwd chat_id. 

In [29]:
cursor = messages_collection.find(
    {"fwd_from.from_id.channel_id": {"$ne": None}}, {"fwd_from.from_id.channel_id": 1}
).limit(5000)

fwd_list = set()
for document in cursor:
    fwd_list.add(document.get("fwd_from", {}).get("from_id", {}).get("channel_id"))
fwd_list = list(fwd_list)
print(len(fwd_list))

date_start = datetime(2016, 1, 1, 0, 0, 0)
date_end = datetime(2024, 1, 1, 0, 0, 0)

start_time = time.time()
cursor_date = messages_collection.find(
    {
        "date": {"$gte": date_start, "$lte": date_end},
        "fwd_from.from_id.channel_id": {"$in": fwd_list},
    }
)
print(f"Messages num: {len(list(cursor_date))}")

end_time = time.time()
elapsed_time = end_time - start_time
print(
    f"Time taken to list() 15 chats based on datetime criteria: {elapsed_time} seconds"
)

77
Messages num: 5004
Time taken to list() 15 chats based on datetime criteria: 1.7020316123962402 seconds


In [30]:
start_time = time.time()

cursor_date = messages_collection.find(
    {"fwd_from.from_id.channel_id": {"$in": fwd_list}}
)

print(f"Messages num: {len(list(cursor_date))}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken to list() 2024: {elapsed_time} seconds")

Messages num: 5014
Time taken to list() 2024: 1.52162504196167 seconds


Next, let's talk about sorting. We can see in one of the previous cells about indices this information:

    SON([('v', 2), ('key', SON([('date', 1), ('peer_id.channel_id', 1)])), ('name', 'date_1_peer_id.channel_id_1')])


"SON([('date', 1)" means that our date is sorted in ascending order. If there would be number -1, it would mean descending. 

The choise of ascending/descending on a single field, means nothing, as MongoDB can traverse the index in either direction. Nevertheless, it means a lot for compund index. as Mongo db will first sort the first field, and then for each first field it will sort the second one. In this case of compund index on a date and chat_id field, it will sort first the date, and then in each date, it will sort chat_id. This does not make a lot of sense, as our date field is very unique due to time. Nevertheless, It works really well, although I am not sure if it would be even faster if we would have two separate indices. I can assume that as db grows, the choise of wheteher to use compound index or not, will depend on a use case. 

Furthermore, we could definitely define another field named date, thus we will not have that many unique values.

### Create a single index on an existing field

Now, we can see that our channel_id from fwd_from.from_id.channel_id works much better! Even better than the compound index!

Be cautious with index creation as this can slow down writing process, as well as writing. Therefore, 

In [20]:
start_time = time.time()
messages_collection.create_index("fwd_from.from_id.channel_id")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken to create index: {elapsed_time} seconds")

Time taken to create index: 265.11340498924255 seconds


In [26]:
start_time = time.time()
cursor = messages_collection.find(
    {"fwd_from.from_id.channel_id": {"$ne": None}}, {"fwd_from.from_id.channel_id": 1}
).limit(2000)


for document in cursor:
    fwd_list.append(document.get("fwd_from", {}).get("from_id", {}).get("channel_id"))

end_time = time.time()
elapsed_time = end_time - start_time
print(
    f"Time taken to list() 10000 chats ids from indexed field: {elapsed_time} seconds"
)

fwd_list = list(set(fwd_list))
print(len(fwd_list))

date_start = datetime(2014, 1, 1, 0, 0, 0)
date_end = datetime(2024, 1, 1, 0, 0, 0)

start_time = time.time()
cursor_date = messages_collection.find(
    {
        "date": {"$gte": date_start, "$lte": date_end},
        "fwd_from.from_id.channel_id": {"$in": fwd_list},
    }
)
print(f"Messages num: {len(list(cursor_date))}")

end_time = time.time()
elapsed_time = end_time - start_time
print(
    f"Time taken to list() 15 chats based on datetime criteria: {elapsed_time} seconds"
)

Time taken to list() 10000 chats ids from indexed field: 0.06733179092407227 seconds
55
Messages num: 2315
Time taken to list() 15 chats based on datetime criteria: 0.7402799129486084 seconds


Finally, lets compare compound index vs two single indices.

In [27]:
date_start = datetime(2020, 1, 1, 0, 0, 0)
date_end = datetime(2024, 1, 1, 0, 0, 0)

start_time = time.time()
cursor_date = messages_collection.find(
    {
        "date": {"$gte": date_start, "$lte": date_end},
        "fwd_from.from_id.channel_id": {"$in": fwd_list},
    }
)
print(f"Messages num: {len(list(cursor_date))}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken to list() 2024: {elapsed_time} seconds")

Messages num: 2189
Time taken to list() 2024: 0.5249917507171631 seconds


# 2. Projection

Projection is used to filter how many fields will be retrieved. Here we can see that, whether we use index or not, speed will be affected by the size of transferred data from MongoDB to python, as well. Thus, it is important to use projecion if the document is large.

Here, we have decided to exclude all messages that were not forwarded from other chats, thus we did not utilized index effectively, as we could have filter it more. 

Nevertheless, first, we tried to retrieve all the data based on criteria, and after that: one and two fields respectively. We will use .limit() function to retrieve only first 100,000 matches. 

In [51]:
start_time = time.time()
cursor = messages_collection.find({"fwd_from.from_id.channel_id": {"$ne": None}}).limit(
    100000
)

all_messages = list(cursor)
print(f"Messages num: {len(all_messages)}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken to list() whole message: {elapsed_time} seconds")

Messages num: 100000
Time taken to list() whole message: 61.28468871116638 seconds


In [54]:
for index, message in enumerate(all_messages):
    if index < 5:
        pprint(message, indent=1)

{'_': 'Message',
 '_id': ObjectId('659dd1e5cb4aa618cf047dad'),
 'date': datetime.datetime(2022, 1, 25, 20, 55, 16),
 'edit_date': datetime.datetime(2022, 1, 25, 20, 55, 19),
 'edit_hide': True,
 'entities': [{'_': 'MessageEntityMention', 'length': 17, 'offset': 50},
              {'_': 'MessageEntityUrl', 'length': 66, 'offset': 69}],
 'forwards': 5,
 'from_id': None,
 'from_scheduled': False,
 'fwd_from': {'_': 'MessageFwdHeader',
              'channel_post': 12114,
              'date': datetime.datetime(2022, 1, 25, 13, 4, 46),
              'from_id': {'_': 'PeerChannel', 'channel_id': 1000023516},
              'from_name': None,
              'imported': False,
              'post_author': None,
              'psa_type': None,
              'saved_from_msg_id': None,
              'saved_from_peer': None},
 'grouped_id': None,
 'id': 20887,
 'legacy': False,
 'media': {'_': 'MessageMediaWebPage',
           'webpage': {'_': 'WebPage',
                       'attributes': [],
   

In [59]:
start_time = time.time()
cursor = messages_collection.find(
    {"fwd_from.from_id.channel_id": {"$ne": None}},
    {"message": 1, "fwd_from.from_id.channel_id": 1},
).limit(100000)
messages_subset = list(cursor)
print(f"Messages num: {len(messages_subset)}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken to list() two fields of a message: {elapsed_time} seconds")

Messages num: 100000
Time taken to list() two fields of a message: 2.521620273590088 seconds


In [56]:
for index, message in enumerate(messages_subset):
    if index < 5:
        pprint(message, indent=1)

{'_id': ObjectId('659dd1e5cb4aa618cf047dad'),
 'fwd_from': {'from_id': {'channel_id': 1000023516}},
 'message': 'Cómo hacer un sistema hidropónico NFT casero 🌶🍅\n'
            '@DIY_Hazlotumismo\n'
            '\n'
            'https://ecoinventos.com/como-hacer-sistema-hidroponico-nft-casero/'}
{'_id': ObjectId('659f06efcb4aa618cf18fb6c'),
 'fwd_from': {'from_id': {'channel_id': 1000023516}},
 'message': 'Gigantescos aerogeneradores flotantes de 324 metros capaces de '
            'suministrar energía a 80.000 hogares cada uno.\n'
            '@Futuro_renovable\n'
            '\n'
            'https://ecoinventos.com/wind-catching-systems/'}
{'_id': ObjectId('65a4a0d93a281ea2115121de'),
 'fwd_from': {'from_id': {'channel_id': 1000064590}},
 'message': 'اطلعت على مقطع للأخ دانيال حقيقتجو بعنوان\n'
            'Is Your Zakat Funding the Most Un-Islamic "Islamic" Graduate '
            'School? (Bayan)\n'
            'يُحَذِّرُ فيه من مؤسسة تأخذ من المسلمين في أمريكا أموال صدقات '
      

# 3. Use aggregation, log slow queries, .explain()

There might be some queries where aggregation would help fetch result faster as we could use query operations in parallel.

We can set profiling levels to log info about slow queries.

Also we can utilize .explain() method before running the query. 

Finally, if we query one field more than another, 

In [72]:
explanation = (
    messages_collection.find({"fwd_from.from_id.channel_id": {"$ne": None}})
    .limit(100000)
    .explain()
)

pprint(explanation, indent=1)

{'command': {'$db': 'Telegram-090124-crawler',
             'filter': {'fwd_from.from_id.channel_id': {'$ne': None}},
             'find': 'messages',
             'limit': 100000,
             'singleBatch': True},
 'executionStats': {'allPlansExecution': [],
                    'executionStages': {'closes': 1,
                                        'executionTimeMillisEstimate': 258,
                                        'inputStage': {'closes': 1,
                                                       'collectionScans': 0,
                                                       'collectionSeeks': 100000,
                                                       'executionTimeMillisEstimate': 258,
                                                       'indexScans': 0,
                                                       'indexSeeks': 1,
                                                       'indexesUsed': ['fwd_from.from_id.channel_id_1'],
                                           

In [75]:
explain2 = (
    messages_collection.find(
        {"fwd_from.from_id.channel_id": {"$ne": None}},
        {"message": 1, "fwd_from.from_id.channel_id": 1},
    )
    .limit(100000)
    .explain()
)

pprint(explain2, indent=1)

{'command': {'$db': 'Telegram-090124-crawler',
             'filter': {'fwd_from.from_id.channel_id': {'$ne': None}},
             'find': 'messages',
             'limit': 100000,
             'projection': {'fwd_from.from_id.channel_id': 1, 'message': 1},
             'singleBatch': True},
 'executionStats': {'allPlansExecution': [],
                    'executionStages': {'closes': 1,
                                        'executionTimeMillisEstimate': 408,
                                        'inputStage': {'closes': 1,
                                                       'executionTimeMillisEstimate': 406,
                                                       'inputStage': {'closes': 1,
                                                                      'collectionScans': 0,
                                                                      'collectionSeeks': 100000,
                                                                      'executionTimeMillisEstimate': 3